In [41]:
# using Pkg
# Pkg.add("")

    Updating registry at `~/.julia/registries/General`


LoadError: The following package names could not be resolved:
 * Threads (not found in project, manifest or registry)


In [7]:
using Test
using BenchmarkTools
using LinearAlgebra
using Statistics
using Random


## Include scripts:

In [2]:
include("../../src/util.jl")

compareValues (generic function with 1 method)

## Test Helper

In [3]:
function testHelper(test::Expr; ntests::Integer = 10)
    [eval(test) for i in 1:ntests]
end

testHelper (generic function with 1 method)

## TEST: colCenter!()

### Test - Case 1:

When data is a vector (1 column)...

In [4]:
test1 = quote
    N = 100;
    A = rand([0.0, 1.0], N, 1);
    ## Manually center columns of A
    colMean_A = sum(A)/N;
    true_centered_A = transpose(round.(transpose(A) .- colMean_A; digits = 2));
    colCenter!(A);
    A .= round.(A; digits = 2);
    @test true_centered_A == A;
end;

In [5]:
testHelper(test1; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

### Test - Case 2:

In [6]:
test2 = quote
    N = 100;
    p = rand([1, 5]); # randomly generate the number of rows
    
    A = rand([0.0, 1.0], N, p);
    
    ## Manually center columns of A
    colMean_A = sum(A, dims = 1)./N;
    true_centered_A = round.(A .- colMean_A; digits = 2);
    colCenter!(A);
    A .= round.(A; digits = 2);
    @test true_centered_A == A;
end;

In [7]:
testHelper(test2; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

## TEST: rowCenter!()

When data is a vector, the function should probably throw an exception (makes no sense to center by row mean). We therefore only test the case when there are multiple columns so it makes sense to average the observations in a row.

### Test

In [8]:
test = quote
    N = 100;
    p = rand([2, 5]); # randomly generate the number of rows
    
    A = rand([0.0, 1.0], N, p);
    
    ## Manually center columns of A
    rowMean_A = sum(A, dims = 2)./p;
    true_centered_A = round.(A .- rowMean_A; digits = 2);
    
    rowCenter!(A);
    A .= round.(A; digits = 2);
    
    @test true_centered_A == A;
end;

In [9]:
testHelper(test; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

## TEST: colDivide!()

### Original code:

### Test

In [10]:
test = quote
    N = 100;
    p = rand([1, 5]); # randomly generate the number of rows
    A = rand([0.0, 1.0], N, p);
    
    colVars = mapslices(var, A, dims = 1) |> vec;
    true_weighted_A = transpose(round.(transpose(A) ./ colVars; digits = 2));
    
    colDivide!(A, colVars);
    A = round.(A; digits = 2);
    
    @test true_weighted_A == A;
end;

In [11]:
testHelper(test; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

## TEST: colStandardize!()

### Test:

In [161]:
A = randn(3, 3);
x = [1.0; 2.0; 3.0]

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [162]:
A

3×3 Matrix{Float64}:
 -0.428666  -1.94744    1.10146
  0.390436   0.364148   0.862161
  0.541328  -0.349303  -1.48735

In [163]:
A .* x

3×3 Matrix{Float64}:
 -0.428666  -1.94744    1.10146
  0.780873   0.728296   1.72432
  1.62398   -1.04791   -4.46205

In [164]:
x .* A

3×3 Matrix{Float64}:
 -0.428666  -1.94744    1.10146
  0.780873   0.728296   1.72432
  1.62398   -1.04791   -4.46205

In [165]:
A ./ x

3×3 Matrix{Float64}:
 -0.428666  -1.94744    1.10146
  0.195218   0.182074   0.43108
  0.180443  -0.116434  -0.495783

In [169]:
A .* (1.0 ./ x)

3×3 Matrix{Float64}:
 -0.428666  -1.94744    1.10146
  0.195218   0.182074   0.43108
  0.180443  -0.116434  -0.495783

In [28]:
A = [5;1;2;3;4;0]

6-element Vector{Int64}:
 5
 1
 2
 3
 4
 0

In [29]:
function checkZero1(A::Array{Float64, 1})
    
    test = iszero.(A);
    # sum(test) != 0;
    
end

checkZero1 (generic function with 1 method)

In [30]:
function checkZero2(A::Array{Float64, 1})
    
    for i in 1:length(A)
        
        if A[i] == 0.0
            return true
        end
        
    end
    
    return false

end

checkZero2 (generic function with 1 method)

In [31]:
A = convert(Array{Float64, 1}, A)

6-element Vector{Float64}:
 5.0
 1.0
 2.0
 3.0
 4.0
 0.0

In [32]:
@btime checkZero1(A)

  57.644 ns (2 allocations: 96 bytes)


6-element BitVector:
 0
 0
 0
 0
 0
 1

In [33]:
@btime checkZero2(A)

  9.668 ns (0 allocations: 0 bytes)


true

In [34]:
?@inbounds

```
@inbounds(blk)
```

Eliminates array bounds checking within expressions.

In the example below the in-range check for referencing element `i` of array `A` is skipped to improve performance.

```julia
function sum(A::AbstractArray)
    r = zero(eltype(A))
    for i in eachindex(A)
        @inbounds r += A[i]
    end
    return r
end
```

!!! warning
    Using `@inbounds` may return incorrect results/crashes/corruption for out-of-bounds indices. The user is responsible for checking it manually. Only use `@inbounds` when it is certain from the information locally available that all accesses are in bounds.



In [49]:
1+eps()/2-1

2.220446049250313e-16

In [50]:
eps()/2 + (1-1)

1.1102230246251565e-16

### TT

In [44]:
function tt(v, A)
    
    (n, m) = size(A)
    
    r = similar(A)
    for j = 1:m 
        for i = 1:n
            r[i,j] = v[j] * A[i,j]
        end
    end 
    r
end

tt (generic function with 1 method)

In [45]:
N = 1000;
p = 100; # randomly generate the number of rows
A = rand(N, p);
w = rand(N);

In [46]:
@btime tt(w, A);

  89.368 μs (2 allocations: 781.30 KiB)


In [47]:
@btime A .* w;

  59.760 μs (4 allocations: 781.36 KiB)


In [204]:
@btime w .* A;

  756.669 μs (4 allocations: 7.63 MiB)


In [12]:
test = quote
    N = 100;
    p = rand([1, 5]); # randomly generate the number of rows
    A = rand([0.0, 1.0], N, p);
    
    colMeans = mean(A, dims = 1) |> vec; # mean of each column
    colVars = mapslices(std, A, dims = 1) |> vec; # std of each column
    true_standardized_A = round.(transpose(A) .- colMeans; digits = 2)
    true_standardized_A = transpose(round.(true_standardized_A ./ colVars; digits = 2));
    
    colStandardize!(A);
    A = round.(A; digits = 2);
    
    @test true_standardized_A == A;
end;

In [13]:
testHelper(test; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

## TEST: rowDivide!()

### Test:

In [14]:
test = quote
    N = 100;
    p = rand([2, 5]); # randomly generate the number of rows
    A = rand([0.0, 1.0], N, p);
    
    rowVars = mapslices(var, A, dims = 2) |> vec;
    rowVars = round.(rowVars; digits = 2);
    
    for i in 1:length(rowVars)
        if(rowVars[i] == 0.0)
            rowVars[i] = 1.0
        end
    end
    
    true_weighted_A = round.(A ./ rowVars; digits = 2);
    
    rowDivide!(A, rowVars);
    A = round.(A; digits = 2);
    
    @test true_weighted_A == A;
end;

In [15]:
testHelper(test; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

*** Need to check if dividing by a zero number (0 col or row std)

## TEST: rowMultiply()

### Test - Case1: check if dimensions

In [42]:
N = 100;
p = rand([2, 5]); # randomly generate the number of rows
A = rand([0.0, 1.0], N, p);

x = zeros(N+1)

try 
    rowMultiply(A, x)
catch e
    @test typeof(e) == ErrorException
end

Test Passed

### Test - Case2: check if results match

In [73]:
include("../../src/util.jl");

In [156]:
function colMultiply(A::Matrix{Float64}, x::Vector{Float64})
    
    (n,m) = size(A)
    # if(length(x)!=m)
        # throw(error("Matrix and vector size do not match."))
    # end

    B = similar(A)
    

    # Original
    for j=1:m
        B[:, j] = (@view A[:, j]) * x[j]
    end

    return B
    
    # return A' .* x
    
end

colMultiply (generic function with 1 method)

In [157]:
function colMultiply_2(A::Matrix{Float64}, x::Vector{Float64})
    
    # (n,m) = size(A)
    # if(length(x)!=m)
        # throw(error("Matrix and vector size do not match."))
    # end

    # B = similar(A)
    

    # Original
    # for j=1:m
        # B[:, j] = (@view A[:, j]) * x[j]
    # end

    # return B
    
    return A' .* x
    
end

colMultiply_2 (generic function with 1 method)

In [152]:
test = quote
    N = 1000;
    p = 100;
    # p = rand([2, 5]); # randomly generate the number of rows
    A = rand(N, p);
    
    w = rand(N)
    
    true_weighted_A = round.(A .* w; digits = 2);
    
    test_weighted_A = round.(rowMultiply(A, w); digits = 2)

    
    @test true_weighted_A == test_weighted_A;
end;

In [153]:
testHelper(test; ntests = 1)

1-element Vector{Test.Pass}:
 Test Passed

In [158]:
B = copy(A');

In [159]:
@btime colMultiply(B, w);

  148.271 μs (1002 allocations: 1.62 MiB)


In [160]:
@btime A .* w;

  59.341 μs (4 allocations: 781.39 KiB)


## loop

In [99]:
@btime rowMultiply(A, w); # original

  1.472 ms (2 allocations: 7.63 MiB)


In [70]:
@btime rowMultiply(A, w); # undef

  149.701 μs (2 allocations: 781.33 KiB)


In [66]:
@btime rowMultiply(A, w); # copy

  149.567 μs (2 allocations: 781.33 KiB)


In [100]:
@btime A .* w;

  1.142 ms (4 allocations: 7.63 MiB)


In [98]:
test = quote
    N = 10000;
    p = 10000;
    # p = rand([2, 5]); # randomly generate the number of rows
    A = rand(N, p);
    
    w = rand(N)
    
    true_weighted_A = round.(A .* w; digits = 2);
    
    test_weighted_A = round.(rowMultiply(A, w); digits = 2)

    
    @test true_weighted_A == test_weighted_A;
end;

In [95]:
testHelper(test; ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

In [27]:
get_num_threads()

LoadError: UndefVarError: get_num_threads not defined

In [28]:
test_weighted_A

100×1000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [29]:
true_weighted_A

100×1000 Matrix{Float64}:
 0.28  0.27  0.04  0.13  0.45  0.55  …  0.3   0.41  0.32  0.54  0.0   0.46
 0.01  0.0   0.0   0.0   0.0   0.0      0.01  0.0   0.0   0.0   0.01  0.01
 0.57  0.21  0.54  0.16  0.07  0.57     0.26  0.08  0.67  0.69  0.19  0.66
 0.2   0.16  0.07  0.1   0.01  0.06     0.18  0.11  0.1   0.03  0.24  0.17
 0.0   0.01  0.01  0.01  0.01  0.01     0.0   0.01  0.01  0.01  0.01  0.0
 0.18  0.06  0.24  0.38  0.04  0.28  …  0.09  0.11  0.28  0.32  0.1   0.04
 0.15  0.47  0.28  0.02  0.09  0.3      0.48  0.15  0.27  0.21  0.48  0.36
 0.43  0.01  0.34  0.19  0.25  0.2      0.23  0.4   0.25  0.28  0.45  0.06
 0.61  0.38  0.13  0.03  0.28  0.45     0.34  0.45  0.61  0.39  0.05  0.23
 0.05  0.23  0.09  0.16  0.09  0.16     0.3   0.18  0.32  0.09  0.01  0.3
 0.41  0.33  0.35  0.15  0.63  0.26  …  0.15  0.59  0.32  0.2   0.41  0.08
 0.02  0.22  0.17  0.13  0.14  0.25     0.27  0.15  0.33  0.33  0.08  0.28
 0.45  0.13  0.52  0.33  0.19  0.14     0.21  0.13  0.01  0.42  0.23  0.33
 

### Test - Case3: check if the original is modified

In [30]:
test = quote
    N = 100;
    p = rand([2, 5]); # randomly generate the number of rows
    A = rand([0.0, 1.0], N, p);
    
    w = rand([0.0, 1.0], N)
    
    test_weighted_A = round.(rowMultiply(A, w); digits = 2)
    
    rowDivide!(A, round.(1.0 ./w))

    @test A == test_weighted_A;
end;

In [31]:
testHelper(test; ntests = 10)

Test Failed at In[30]:12
  Expression: A == test_weighted_A
   Evaluated: [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0] == [0.0 0.0 … 0.0 NaN; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … NaN 0.0; 0.0 0.0 … 0.0 0.0]


LoadError: [91mThere was an error during testing[39m

## TEST: shuffleVector()

### Test - output dimension:

In [32]:
test_1 = quote
    N = 100;
    p = 1;
    A = rand([0.0, 1.0], N, p) |> vec; # fixed to be a vector
    rng = MersenneTwister();
    
    result = shuffleVector(rng, A, 5);
    
    
    @test size(result) == (N, 6);
end;

In [33]:
testHelper(test_1, ntests = 1)

1-element Vector{Test.Pass}:
 Test Passed

- Test of sum...

In [34]:
test_2 = quote
    N = 100;
    p = 1;
    A = rand([0.0, 1.0], N, p) |> vec; # fixed to be a vector
    rng = MersenneTwister();
    
    result = shuffleVector(rng, A, 5);
    
    colSums = sum(result, dims = 1) |> vec;
    
    @test (colSums./colSums[1]) == ones(6);
end;

In [35]:
testHelper(test_2, ntests = 10)

10-element Vector{Test.Pass}:
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed
 Test Passed

- Test of pairwise correlations:

In [36]:
test_3 = quote 
    N = 100;
    p = 1;
    A = rand([0.0, 1.0], N, p) |> vec; # fixed to be a vector
    result = shuffleVector(MersenneTwister(), A, 5);
    
    PartCorr_A = tril(cor(result), -1) |> vec;
    
    test_t_vals = mapslices(PartCorr_A) do x
        x * sqrt(98.0/(1.0-x^2))
    end;
    
    histgram(test_t_vals)
end;

## TEST: compareValues()

### Test: